In [13]:
from delta_rest_client import DeltaRestClient, OrderType
import json
import sys
import requests
from time import time
import hmac
import hashlib
from utils import *
import pytz
from indicators import Indicators

key = None
secret = None
with open("creds.json",'r') as f:
    op = json.load(f)
    key = op.get('api')
    secret = op.get('secret')

if key is None or secret is None:
    print(key)
    print(secret)
    sys.exit(1)
baseurl = "https://api.india.delta.exchange"

client = DeltaRestClient(
  base_url=baseurl,
  api_key=key,
  api_secret=secret
)
productId = get_product_id_by_symbol(symbol = 'BTCUSD')
client.set_leverage(productId, 20)

HTTPError: 401 HTTP Error: Unauthorized {"error":{"code":"ip_not_whitelisted_for_api_key","context":{"client_ip":"2405:201:18:833:3c91:f98e:4ac4:8f6d"}},"success":false} for url: https://api.india.delta.exchange/v2/products/27/orders/leverage

In [ ]:
class symbol_management:
    def __init__(self, client, productId):
        self.productId = productId
        self.client = client
        self.qty = 0

    def recordorder(self, entry, stoploss, qty, target, tradetime, tradetype, openorderid):
        data = None
        with open("trades.json", "r") as f:
            data = json.load(f)
        if data.get(tradetime) is not None:
            return 
        data['trade']={
            "tradetime" : tradetime,
            "entry": entry,
            "stoploss": stoploss,
            "target": target,
            "tradetype": tradetype,
            "qty" : qty,
            "openorderid" : openorderid,
        }
        with open("trades.json", "w") as f:
            json.dump(data, f, indent=2)
            print("saved data")
        
    def _cancel_order(self, orderid):
        self.client.cancel_order(productId, orderid)
    
    def print_position_state(self):
        print("product id : ", self.productId)
        print("isactiveposition : ", self.isactiveposition)
        print("stoplossid : ", self.stoplossid)
        print("targetid : ",self.targetid)
        print("opentrade : ", self.opentradeid)
        print("myposition state : ", self.activeTradeState)    
    
    def trade(self, entry, stoploss, trade_type):
        r = self.client.get_margined_position(self.productId)
        if r is None:
            r =  self.client.get_live_orders()
            print('cancelling orders before plaing trade')
            for orders in r:
                if  (orders['state'] == 'pending' or orders['state'] == 'open') and orders['product_id'] == self.productId:
                    self._cancel_order(orders['id'])
            order = self._place_order(tradetype, entry, self.qty)
            print("Limit order placed : " , limitorderqty)
        pnl = r['realized_pnl']
        activesize = r['size']
        else:
            if (activesize <0 and tradetype == "sell") and (activesize >0 and tradetype == "buy"):
                print("ignore this trade as it is unsafe to place [trade don not sync]")
                return 
            if pnl > 0:
                self.qty = min(2,self.qty - 1)
            if pnl < 0:
                self.qty = min(2,self.qty - 1)
            limitorderqty = self.qty + activesize
            self._place_order(tradetype, entry, limitorderqty)
            print("Limit order placed : " , limitorderqty)
        

    def _place_order(self, tradetype, entry, qty):
        if self._is_safe_placing():
            order = self.client.place_order(self.productId, qty, tradetype, limit_price=entry, time_in_force=None, order_type=OrderType.LIMIT)
            return order
        return None

In [ ]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import pytz
symbolmanager = symbol_management(client, productId)

class Strategy:
    def __init__(self, symbol, rr = 1.3, timer = 1, interval = '1m', tick = 3):
        self.symbol = symbol
        self.rr = rr
        self.timer = timer
        self.interval = interval

    def _get_unix_timestamp(self, dt):
        return int(time.mktime(dt.timetuple()))

    def _load_historical_data(self,days=1):
        interval = self.interval
        end_dt = datetime.now() + timedelta(minutes=1)
        start_dt = end_dt - timedelta(days=days)

        start_unix = self._get_unix_timestamp(start_dt)
        end_unix = self._get_unix_timestamp(end_dt)

        url = 'https://api.india.delta.exchange/v2/history/candles'
        params = {
            'resolution': interval,
            'symbol': self.symbol,
            'start': start_unix,
            'end': end_unix
        }
        headers = {'Accept': 'application/json'}
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['result'])
            df['time'] = pd.to_datetime(df['time'], unit='s',  utc=True).dt.tz_convert(pytz.timezone('Asia/Kolkata'))
            df = df[::-1]
            return df
        else:
            raise ConnectionError(f"Failed to fetch data: {response.status_code} - {response.text}")

    def run_strategy(self):
        try:
            df = self._load_historical_data()
            df = Indicators.supertrend(df, 15,1, 'supertrend')
            df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
            df['buy'] = (df['supertrend_15_1_direction'].shift(1)==-1) & (df['supertrend_15_1_direction']==1)
            df['sell'] = (df['supertrend_15_1_direction'].shift(1)==1) & (df['supertrend_15_1_direction']==-1)
            lastminute = datetime.strptime(df.iloc[-1]['time'], "%Y-%m-%d %H:%M:%S").minute
            currentminute = datetime.today().minute
            df = df.tail(1)
            return 
            if row['buy']:
                entry = row['close']
                symbolmanager.trade(entry + 5,'buy')

            if row['sell']:
                entry = row['close']
                symbolmanager.trade(entry - 5, 'sell')

        except Exception as e:
            print(f"Error running strategy: {e}")

    def start(self):
        lasttradedminute = None
        while(True):
            if datetime.today().minute != lasttradedminute:
                self.run_strategy()
                lasttradedminute = datetime.today().minute
                time.sleep(5)
            else:
                time.sleep(5)


    def stop(self):
        self.running = False

strategy = Strategy('BTCUSD')
strategy.start()

       close      high       low      open                 time  volume  \
29  119585.5  119688.5  119553.0  119633.0  2025-07-16 22:57:00   12075   
28  119584.5  119609.0  119549.5  119586.5  2025-07-16 22:58:00    9008   
27  119610.0  119636.5  119581.5  119583.5  2025-07-16 22:59:00    4172   
26  119607.5  119628.5  119564.0  119617.0  2025-07-16 23:00:00    5771   
25  119626.0  119638.0  119591.0  119607.5  2025-07-16 23:01:00    3798   
24  119630.0  119630.0  119591.5  119625.0  2025-07-16 23:02:00   11064   
23  119593.5  119631.0  119591.5  119630.0  2025-07-16 23:03:00    3024   
22  119629.5  119629.5  119542.0  119594.5  2025-07-16 23:04:00    9061   
21  119625.0  119662.0  119623.5  119628.5  2025-07-16 23:05:00    7014   
20  119571.0  119652.5  119571.0  119630.5  2025-07-16 23:06:00    4499   
19  119494.0  119589.5  119473.5  119573.5  2025-07-16 23:07:00   11842   
18  119516.0  119526.0  119450.0  119495.0  2025-07-16 23:08:00    6042   
17  119511.0  119544.5  1

KeyboardInterrupt: 

In [12]:
import pandas_ta as ta

In [25]:
client.get_position(productId)

{'entry_price': '119362.00000000', 'size': 1}

In [36]:
r = client.get_margined_position(productId)
print(r['size'])

-1
